In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = pd.read_csv('f1_2019_to_2022_all_drivers_all_data.csv',low_memory=False) # it's for 2019 to 2022 <- Train 

#Change to numerical values
df['LapTime'] = pd.to_timedelta(df['LapTime']).dt.total_seconds()


columns_to_check = ['Position', 'IsPersonalBest', 'TrackStatus']
df = df.dropna(subset=columns_to_check)


# Change strings to integer
df['Sector1Time'] = pd.to_timedelta(df['Sector1Time']).dt.total_seconds()
df['Sector2Time'] = pd.to_timedelta(df['Sector2Time']).dt.total_seconds()
df['Sector3Time'] = pd.to_timedelta(df['Sector3Time']).dt.total_seconds()


df['Rainfall'] = df['Rainfall'].astype(int) # Encode Rainfall To integer
df['FreshTyre'] = df['FreshTyre'].astype(int) # Encode FreshTyre To integer
df['IsAccurate'] = df['IsAccurate'].astype(int)
df["IsPersonalBest"] = df['IsPersonalBest'].astype(int)



#Drop non-related features 
columns_to_drop = [
    'Time', 'LapStartTime',
    'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
    'PitOutTime', 'PitInTime',
    'LapStartDate', 'Deleted', 'DeletedReason',
    'FastF1Generated'
]
df = df.drop(columns=columns_to_drop)



# TO use imputation method
df['Original_Driver'] = df['Driver']
df['Original_Circuit'] = df['Circuit']


#One hot encoding
df = pd.get_dummies(df, columns=['Driver', 'Circuit','Compound','Team'])

# In case of Standardization
numeric_features = ['SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'AirTemp', 'Humidity', 'Pressure', 'TrackTemp', 'WindDirection', 'WindSpeed',]


df = df.dropna(subset=['LapTime'])

X = df.drop(['LapTime'], axis=1) # Exclude response variable ('LapTime') 

y = df['LapTime']


print(X.info(verbose=True))




<class 'pandas.core.frame.DataFrame'>
Int64Index: 80808 entries, 0 to 82417
Data columns (total 110 columns):
 #    Column                               Dtype  
---   ------                               -----  
 0    DriverNumber                         int64  
 1    LapNumber                            float64
 2    Stint                                float64
 3    Sector1Time                          float64
 4    Sector2Time                          float64
 5    Sector3Time                          float64
 6    SpeedI1                              float64
 7    SpeedI2                              float64
 8    SpeedFL                              float64
 9    SpeedST                              float64
 10   IsPersonalBest                       int64  
 11   TyreLife                             float64
 12   FreshTyre                            int64  
 13   TrackStatus                          float64
 14   Position                             float64
 15   IsAccurate       

In [2]:
# It could be deleted if I use RandomForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
scaler = StandardScaler()

X[numeric_features] = scaler.fit_transform(X[numeric_features])


In [3]:
# Check for NaN values in each column
nan_counts = X.isna().sum()

# Print columns with NaN count more than 0
print(nan_counts[nan_counts > 0])


Sector1Time     1507
Sector2Time       28
Sector3Time       31
SpeedI1        11665
SpeedI2           77
SpeedFL         2510
SpeedST         6268
dtype: int64


In [4]:

means_sector1 = X.groupby(['Original_Driver', 'Original_Circuit'])['Sector1Time'].mean()

# Map the means back to the original dataframe to fill missing values
X['Sector1Time'] = X.apply(lambda row: means_sector1[(row['Original_Driver'], row['Original_Circuit'])] if pd.isnull(row['Sector1Time']) else row['Sector1Time'], axis=1)

#  for Sector2Time
means_sector2 = X.groupby(['Original_Driver', 'Original_Circuit'])['Sector2Time'].mean()

X['Sector2Time'] = X.apply(lambda row: means_sector2[(row['Original_Driver'], row['Original_Circuit'])] if pd.isnull(row['Sector2Time']) else row['Sector2Time'], axis=1)

#  for Sector3Time
means_sector3 = X.groupby(['Original_Driver', 'Original_Circuit'])['Sector3Time'].mean()
X['Sector3Time'] = X.apply(lambda row: means_sector3[(row['Original_Driver'], row['Original_Circuit'])] if pd.isnull(row['Sector3Time']) else row['Sector3Time'], axis=1)

#  for SpeedI1
means_speedI1 = X.groupby(['Original_Driver', 'Original_Circuit'])['SpeedI1'].mean()
X['SpeedI1'] = X.apply(lambda row: means_speedI1[(row['Original_Driver'], row['Original_Circuit'])] if pd.isnull(row['SpeedI1']) else row['SpeedI1'], axis=1)


means_speedI2 = X.groupby(['Original_Driver', 'Original_Circuit'])['SpeedI2'].transform('mean')
X['SpeedI2'] = X['SpeedI2'].fillna(means_speedI2)

#  for SpeedFL
means_speedFL = X.groupby(['Original_Driver', 'Original_Circuit'])['SpeedFL'].transform('mean')
X['SpeedFL'] = X['SpeedFL'].fillna(means_speedFL)

#  for SpeedST
means_speedST = X.groupby(['Original_Driver', 'Original_Circuit'])['SpeedST'].transform('mean')
X['SpeedST'] = X['SpeedST'].fillna(means_speedST)


In [5]:
# Check for NaN values in each column
nan_counts = X.isna().sum()

# Print columns with NaN count more than 0
print(nan_counts[nan_counts >0])


Sector1Time    3
SpeedFL        1
dtype: int64


In [6]:

print(len(X), len(y))

80808 80808


In [7]:
# After attempting to fill NaNs with group means
X['Sector1Time'].fillna(X['Sector1Time'].mean(), inplace=True)
X['Sector2Time'].fillna(X['Sector2Time'].mean(), inplace=True)
X['Sector3Time'].fillna(X['Sector3Time'].mean(), inplace=True)
X['SpeedI1'].fillna(X['SpeedI1'].mean(), inplace=True)
X['SpeedI2'].fillna(X['SpeedI2'].mean(), inplace=True)
X['SpeedFL'].fillna(X['SpeedFL'].mean(), inplace=True)
X['SpeedST'].fillna(X['SpeedST'].mean(), inplace=True)


# Check for NaN values in each column
nan_counts = X.isna().sum()

# Print columns with NaN count more than 0
print(nan_counts[nan_counts >0])

Series([], dtype: int64)


In [8]:

print(len(X), len(y))



80808 80808


In [9]:
from sklearn.decomposition import PCA

# Assuming df_encoded already excludes 'LapTime' and has standardized numeric features and one-hot encoded categorical variables

#Deleted String Columns
X = X.drop(['Original_Driver', 'Original_Circuit'], axis= 1)

# Initialize PCA, choose the number of components e.g., 95% of variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X)

# Explained variance ratio can be helpful to understand the coverage
print(f"Explained Variance Ratio: {pca.explained_variance_ratio_.sum()}")



Explained Variance Ratio: 0.9713348251582974


In [10]:
nan_counts = y.isna().sum()

# Print columns with NaN count more than 0
print(nan_counts[nan_counts >0])

[]


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100,random_state=42)

model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [13]:
from sklearn.metrics import mean_squared_error

predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = mse ** 0.5
print(f"RMSE: {rmse}")


RMSE: 4.827832652188716
